# Geographic visualization

## This Script Contains the following

### 1.Import data and libraries

### 2. Data Wrangling

### 3. Data Cleaning

### 4. Plotting a Choropleth

## 1. Import data and libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

### Import the JSON file

In [3]:
# Import ".json" file for the U.S. 
country_geo = r"C:\Users\tjsra\OneDrive\Desktop\CF data analyst\Achievement 6\02 Data\Original Data\us-states.json"


In [4]:
with open(country_geo, 'r') as f:
    geo_data = json.load(f)

### Import the dataset

In [5]:
import_path = r"C:\Users\tjsra\OneDrive\Desktop\CF data analyst\Achievement 6\02 Data\prepared data\Conditions_Contributing_to_COVID-19_Deaths_Cleaned.csv"

In [6]:
df = pd.read_csv(import_path)

In [7]:
df.head()

,Data As Of,Start Date,End Date,Year,Month,State,Condition Group,Condition,ICD10_codes,Age Group,COVID-19 Deaths,Number of Mentions,Flag
0,2023-09-24,2020-01-01,2020-01-31,2020.0,1.0,Alabama,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.0,0.0,NaN
1,2023-09-24,2020-02-01,2020-02-29,2020.0,2.0,Alabama,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.0,0.0,NaN
2,2023-09-24,2020-03-01,2020-03-31,2020.0,3.0,Alabama,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.0,0.0,NaN
3,2023-09-24,2020-04-01,2020-04-30,2020.0,4.0,Alabama,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.0,0.0,NaN
4,2023-09-24,2020-05-01,2020-05-31,2020.0,5.0,Alabama,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,0.0,0.0,NaN


In [8]:
df.shape

(430560, 13)

## 2. Data Wrangling

In [9]:
# Extract state names from the dataset
dataset_states = set(df['State'].unique())


In [10]:
# Extract state names from the GeoJSON file
geojson_states = set(feature['properties']['name'] for feature in geo_data['features'])


In [11]:
# Compare the state names
missing_in_geojson = dataset_states - geojson_states
missing_in_dataset = geojson_states - dataset_states

In [12]:
print(f"States in dataset but not in GeoJSON: {missing_in_geojson}")
print(f"States in GeoJSON but not in dataset: {missing_in_dataset}")

States in dataset but not in GeoJSON: {'District of Columbia', 'New York City'}
States in GeoJSON but not in dataset: set()


### To fix the discrepancies

In [13]:
# Normalize state names in the dataset
df['State'] = df['State'].str.strip().str.title()


In [14]:
# Create a mapping dictionary for the missing states
state_mapping = {
    'District of Columbia': 'District of Columbia',
    'New York City': 'New York'
}

In [15]:
# Apply the mapping to the dataset
df['State'] = df['State'].replace(state_mapping)


In [16]:
# Re-check for discrepancies
dataset_states = set(df['State'].unique())
geojson_states = set(feature['properties']['name'] for feature in geo_data['features'])


In [17]:
missing_in_geojson = dataset_states - geojson_states
missing_in_dataset = geojson_states - dataset_states

In [18]:
print(f"States in dataset but not in GeoJSON: {missing_in_geojson}")
print(f"States in GeoJSON but not in dataset: {missing_in_dataset}")

States in dataset but not in GeoJSON: {'District Of Columbia'}
States in GeoJSON but not in dataset: set()


In [19]:
# Remove "District Of Columbia" from the dataset
df = df[df['State'] != 'District Of Columbia']


In [20]:
# Re-check for discrepancies
dataset_states = set(df['State'].unique())
missing_in_geojson = dataset_states - geojson_states
missing_in_dataset = geojson_states - dataset_states


In [21]:
print(f"States in dataset but not in GeoJSON: {missing_in_geojson}")
print(f"States in GeoJSON but not in dataset: {missing_in_dataset}")

States in dataset but not in GeoJSON: set()
States in GeoJSON but not in dataset: set()


### Merge the data with JSON file.

In [22]:
# Extract state names from the GeoJSON file
geojson_states = [feature['properties']['name'] for feature in geo_data['features']]

# Create a DataFrame from the GeoJSON states
geo_df = pd.DataFrame({'State': geojson_states})

# Merge the dataset with the GeoJSON DataFrame
merged_df = pd.merge(df, geo_df, on='State', how='inner')

## 3. Check for consistency

In [23]:
# Check for missing states after merge
missing_in_merged = set(df['State']) - set(merged_df['State'])

print(f"States in original dataset but not in merged dataset: {missing_in_merged}")

States in original dataset but not in merged dataset: set()


In [24]:
merged_df.isnull().sum()

Data As Of                 0
Start Date                 0
End Date                   0
Year                       0
Month                      0
State                      0
Condition Group            0
Condition                  0
ICD10_codes                0
Age Group                  0
COVID-19 Deaths            0
Number of Mentions         0
Flag                  273574
dtype: int64

In [25]:
# Fill missing values in the 'Flag' column with 0
merged_df['Flag'].fillna(0, inplace=True)

C:\Users\tjsra\AppData\Local\Temp\ipykernel_14956\3541593458.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Flag'].fillna(0, inplace=True)


In [26]:
print(merged_df.isnull().sum())

Data As Of            0
Start Date            0
End Date              0
Year                  0
Month                 0
State                 0
Condition Group       0
Condition             0
ICD10_codes           0
Age Group             0
COVID-19 Deaths       0
Number of Mentions    0
Flag                  0
dtype: int64


In [27]:
# Check for duplicate rows
duplicates = merged_df[merged_df.duplicated()]

# Print the duplicate rows
print(duplicates)

        Data As Of  Start Date    End Date    Year  Month     State  \
264960  2023-09-24  2020-01-01  2020-01-31  2020.0    1.0  New York   
264961  2023-09-24  2020-02-01  2020-02-29  2020.0    2.0  New York   
264966  2023-09-24  2020-07-01  2020-07-31  2020.0    7.0  New York   
264967  2023-09-24  2020-08-01  2020-08-31  2020.0    8.0  New York   
264968  2023-09-24  2020-09-01  2020-09-30  2020.0    9.0  New York   
...            ...         ...         ...     ...    ...       ...   
273105  2023-09-24  2020-01-01  2020-01-31  2020.0    1.0  New York   
273106  2023-09-24  2020-02-01  2020-02-29  2020.0    2.0  New York   
273150  2023-09-24  2020-01-01  2020-01-31  2020.0    1.0  New York   
273195  2023-09-24  2020-01-01  2020-01-31  2020.0    1.0  New York   
273196  2023-09-24  2020-02-01  2020-02-29  2020.0    2.0  New York   

             Condition Group                Condition ICD10_codes Age Group  \
264960  Respiratory diseases  Influenza and pneumonia     J09-J18   

In [28]:
# Remove duplicate rows
merged_df = merged_df.drop_duplicates()

In [29]:
# Verify that duplicates are removed
print(f"Number of duplicate rows after removal: {merged_df.duplicated().sum()}")

Number of duplicate rows after removal: 0


##  4. Plotting a choropleth map

In [30]:
# Create the Choropleth map
m = folium.Map(location=[37.8, -96.9], zoom_start=4)

folium.Choropleth(
    geo_data=geo_data,
    name='choropleth',
    data=merged_df,
    columns=['State', 'COVID-19 Deaths'],  # Adjust this to the column you want to visualize
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='COVID-19 Deaths'
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)


In [31]:
map_path = r"C:\Users\tjsra\OneDrive\Desktop\CF data analyst\Achievement 6\04 Analysis\choropleth_map.html"
m.save(map_path)

In [32]:
import pandas as pd
import json
import folium
from IPython.display import IFrame
import os

In [34]:
# Display the map in the Jupyter Notebook
from IPython.display import IFrame
IFrame(map_path, width=800, height=600)

In [35]:
print(map_path)  # Verify the path

C:\Users\tjsra\OneDrive\Desktop\CF data analyst\Achievement 6\04 Analysis\choropleth_map.html


## 5. Answer to Existing Research Questions.

###  Which states have the highest and lowest prevalence of COVID-19 deaths?
###  The choropleth map clearly shows that California, Florida, and Texas have the highest COVID-19 death rates, indicated by the darkest red color on the map. States like Alaska, Vermont, and Wyoming have the lowest COVID-19 death rates, indicated by the lightest color on the map.

## New Reaearh Questions.

###  How does the age distribution in each state affect the COVID-19 death rates?

###  What is the role of healthcare infrastructure of each states in COVID-19 death rates?